In [ ]:
import pickle
from collections import namedtuple
from tqdm import tqdm_notebook as tqdm

import pandas as pd
import spacy
from spacy_cld import LanguageDetector


In [ ]:
nlp = spacy.load('en_core_web_sm')
language_detector = LanguageDetector()
nlp.add_pipe(language_detector)

In [ ]:
# Util function to write lazy chunks back to disk
output_columns = ["id", "hash1", "hash2", "user", "user_pic", "channel_url", "channel_id", "comment", 
                  "depth", "timestamp", "language"]
comments_path = "data/comments_language.csv"
def write_to_disk(chunk):
    with open(comments_path, "a", encoding="utf-8") as file:
        chunk.to_csv(file, index=False, header=False)

# Lazy data reader into DataFrame
transcripts_reader = pd.read_csv("data/comments.csv", chunksize=500, names=output_columns[:-1])
# Reader to pick up where we ended
completed_reader = pd.read_csv("data/comments_language.csv", chunksize=500, names=output_columns)

In [9]:
# Run this to move the transcripts_reader iterator to where we stopped last time
for _ in completed_reader:
    next(transcripts_reader)

In [ ]:
for ix, transcripts in enumerate(tqdm(transcripts_reader)):
    languages = []
    for ix, transcript in transcripts.iterrows():
        content = str(transcript[7])
        doc = nlp(content)
        try:
            language = doc._.languages[0]
        except IndexError:
            language = "?"
        languages.append(language)
    transcripts["language"] = languages
    write_to_disk(transcripts)

In [10]:
for ix, transcripts in enumerate(tqdm(transcripts_reader)):
    pass